In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import logging
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk

In [2]:
logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

In [3]:
def load_images(folder_path):
    logging.info("loading images...")
    images=[]
    labels=[]
    classes=os.listdir(folder_path)

    for label,class_name in enumerate(classes):
        class_path=os.path.join(folder_path,class_name)
        if not os.path.isdir(class_path):
            logging.warning(f"this file is not folder:{class_path}")
            continue

        for image_file in os.listdir(class_path):
            image_path=os.path.join(class_path,image_file)
            try:
                image=cv2.imread(image_path,cv2.IMREAD_COLOR)
                if image is not None:
                    image=cv2.resize(image,(64,64))
                    image=image.flatten() / 255
                    images.append(image)
                    labels.append(label)
                else:
                    logging.warning(f"cant open image from:{image_path}")
            except Exception as e:
                logging.error(f"cant process from:{image_path}")
    logging.info("loading images are finished")
    return np.array(images),np.array(labels), classes

        

In [4]:
data_folder=r"C:\KNN Animal Classifier"
logging.info("loading image from data folder")
X,y,class_names=load_images(data_folder)
logging.info("completed")
logging.info(f"number of samples:{len(X)} class names{class_names}")

 2025-01-12 09:15:04,270 - INFO - loading image from data folder
 2025-01-12 09:15:04,271 - INFO - loading images...
 2025-01-12 09:15:08,940 - WARNING - this file is not folder:C:\KNN Animal Classifier\New Text Document.txt
 2025-01-12 09:15:08,940 - INFO - loading images are finished
 2025-01-12 09:15:08,966 - INFO - completed
 2025-01-12 09:15:08,966 - INFO - number of samples:888 class names['cat', 'dog', 'New Text Document.txt']


In [5]:
logging.info("splitting data")
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
logging.info(f"training samples: {len(X_train)}, testing samples: {len(X_test)}")

 2025-01-12 09:15:10,118 - INFO - splitting data
 2025-01-12 09:15:10,140 - INFO - training samples: 710, testing samples: 178


In [6]:
logging.info("training AI:...")
knn=KNeighborsClassifier(n_neighbors=2)
knn=knn.fit(X_train,y_train)
logging.info("finished")

 2025-01-12 09:15:12,466 - INFO - training AI:...
 2025-01-12 09:15:12,477 - INFO - finished


In [7]:
logging.info("testing AI accuracy...")
y_pred = knn.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
logging.info(f"AI accuracy:{accuracy*100:.2f}")

 2025-01-12 09:15:17,646 - INFO - testing AI accuracy...
 2025-01-12 09:15:17,780 - INFO - AI accuracy:70.79


In [8]:
def predict_new_image(image_path):
    try:
        image=cv2.imread(image_path,cv2.IMREAD_COLOR)

        
        if image is None:
            logging.error("cant process image")
            return None

            
        image=cv2.resize(image,(64,64))
        image=image.flatten() /255
        prediction=knn.predict([image])
        logging.info(f"finished,image is {class_names[prediction[0]]}")
        return class_names[prediction[0]]

        
    except Exception as e:
        logging.error(f"cant open {image}")
        return None

    

In [9]:
#predict_new_image(r"C:\KNN Test\cat\1.jpg")

In [10]:
def predict_image_gui(image_path):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if img is None:
            return "invalid Image"
        img = cv2.resize(img, (64, 64)).flatten() / 255.0
        prediction = knn.predict([img])
        return class_names[prediction[0]]
    except Exception as e:
        logging.error(f"Error in GUI prediction: {e}")
        return "error"

def select_image():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return


    image = Image.open(file_path) 
    image = image.resize((300, 300))  
    img_tk = ImageTk.PhotoImage(image)


    img_label.config(image=img_tk)
    img_label.image = img_tk


    prediction = predict_image_gui(file_path)
    result_label.config(text=f"Animal: {prediction}")



root = tk.Tk()
root.title("Animal Classifier")
root.geometry("400x600")
btn = Button(root, text="Select Image", command=select_image, bg="blue", fg="white", font=("Arial", 12))
btn.pack(pady=10)
img_label = Label(root)
img_label.pack()
result_label = Label(root, text="Animal: ", font=("Arial", 14))
result_label.pack(pady=20)
root.mainloop()